<a href="https://colab.research.google.com/github/rrfsantos/Desafios-NLP/blob/main/Desafio-NLP--NER/insert_label_iob_convert_spacy_arquivos_separados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --user -U pip setuptools wheel
!pip install --user -U spacy[transformers,lookups]
!python -m spacy download en_core_web_lg

In [ ]:
import pandas as pd
import json
import warnings
import csv
import spacy
import numpy as np
import re

from spacy.lang.en import English
from spacy.pipeline import EntityRuler

warnings.filterwarnings("ignore")

In [ ]:
!python -m spacy info

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
workdir_path = '/content/drive/My Drive/desafio 2/'
os.chdir(workdir_path)

## Ler arquivos IOB e incluir o nome da classe à tag

In [ ]:
dirs = ['s800','linnaeus','BC5CDR-disease','NCBI-disease','BC2GM','JNLPBA','BC4CHEMD','BC5CDR-chem']

### Arquivos de treino

In [ ]:
for dir in dirs:
    df = pd.read_csv('NERdata/' + dir + '/train_dev.tsv', sep='\t', header=None, names=['token','tag'],
                         quoting=csv.QUOTE_NONE, error_bad_lines=False, skipinitialspace = True)
       
    if (dir == 's800' or dir == 'linnaeus'):
      df['tag'] = df['tag'].replace(['B','I'],['B-SPECIES','I-SPECIES'])
    elif (dir == 'BC5CDR-disease' or dir == 'NCBI-disease'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DISEASE','I-DISEASE'])
    elif (dir == 'BC2GM' or dir == 'JNLPBA'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DRUG_PROTEIN','I-DRUG_PROTEIN'])
    else:
      df['tag'] = df['tag'].replace(['B','I'],['B-CHEMICALS','I-CHEMICALS'])
  
    df.dropna(inplace=True)

    df.to_csv('IOBdata/iob_train_' + dir + '.iob', sep='\t', index=False, header=False)  

### Arquivos de Teste

In [ ]:
for dir in dirs:
    df = pd.read_csv('NERdata/' + dir + '/test.tsv', sep='\t', header=None, names=['token','tag'],
                         quoting=csv.QUOTE_NONE, error_bad_lines=False, skipinitialspace = True)
     

    if (dir == 's800' or dir == 'linnaeus'):
      df['tag'] = df['tag'].replace(['B','I'],['B-SPECIES','I-SPECIES'])
    elif (dir == 'BC5CDR-disease' or dir == 'NCBI-disease'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DISEASE','I-DISEASE'])
    elif (dir == 'BC2GM' or dir == 'JNLPBA'):
      df['tag'] = df['tag'].replace(['B','I'],['B-DRUG_PROTEIN','I-DRUG_PROTEIN'])
    else:
      df['tag'] = df['tag'].replace(['B','I'],['B-CHEMICALS','I-CHEMICALS'])

    df.dropna(inplace=True)

    df.to_csv('IOBdata/iob_test_' + dir + '.iob', sep='\t', index=False, header=False)  

## Converter arquivos em formato IOB para o formato binário .spacy

### Arquivos de Treino

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_train_s800.iob ./SPACYdata/train
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_s800.iob ./SPACYdata/train

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_train_linnaeus.iob ./SPACYdata/train
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_linnaeus.iob ./SPACYdata/train

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_train_BC5CDR-disease.iob ./SPACYdata/train
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_BC5CDR-disease.iob ./SPACYdata/train

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_train_NCBI-disease.iob ./SPACYdata/train
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_NCBI-disease.iob ./SPACYdata/train

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_train_BC2GM.iob ./SPACYdata/train
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_BC2GM.iob ./SPACYdata/train

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_train_JNLPBA.iob ./SPACYdata/train
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_JNLPBA.iob ./SPACYdata/train

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_train_BC5CDR-chem.iob ./SPACYdata/train
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_BC5CDR-chem.iob ./SPACYdata/train

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_train_BC4CHEMD.iob ./SPACYdata/train
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_train_BC4CHEMD.iob ./SPACYdata/train

### Arquivos de teste

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_test_s800.iob ./SPACYdata/test
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_s800.iob ./SPACYdata/test

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_test_linnaeus.iob ./SPACYdata/test
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_linnaeus.iob ./SPACYdata/test

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_test_BC5CDR-disease.iob ./SPACYdata/test
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_BC5CDR-disease.iob ./SPACYdata/test

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_test_NCBI-disease.iob ./SPACYdata/test
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_NCBI-disease.iob ./SPACYdata/test

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_test_BC2GM.iob ./SPACYdata/test
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_BC2GM.iob ./SPACYdata/test

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_test_JNLPBA.iob ./SPACYdata/test
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_JNLPBA.iob ./SPACYdata/test

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_test_BC5CDR-chem.iob ./SPACYdata/test
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_BC5CDR-chem.iob ./SPACYdata/test

In [ ]:
!python -m spacy convert -c iob -s -n 10 -t json IOBdata/iob_test_BC4CHEMD.iob ./SPACYdata/test
!python -m spacy convert -c iob -s -n 10 IOBdata/iob_test_BC4CHEMD.iob ./SPACYdata/test